In [46]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
import re
import numpy as np
import mysql.connector
from mysql.connector import Error

In [64]:
#the function determined if the url enter is accurate
def get_source(url):

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [65]:
#This part gets all anchors in the website. The scraper will avoid Google ads and all unwanted Google links
def scrape_google(query):

    query = urllib.parse.quote_plus(query)
    response = get_source("https://www.google.com/search?q=" + query)

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://www.googleadservices.'
                      'https://maps.google.')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)
   

    return links

In [66]:
#Searchers to seek
plants_search = ['house plants', 'green plants', 'plants','hard to kill plants', 'low light plants', \
              'how to take care of plants', 'cacti', 'easy plants', 'outdoor plants', 'winter plants',\
             'summer plants', 'all year round plants', 'flowering plants', 'wall plants', \
                 'green plants', 'easy house plants', 'easy to care plants']

In [67]:
#Appending each outcome list into a list
results = []
for query in plants_search:
    result = scrape_google(query)
    results.append(result)


In [68]:
#making a list of nested list into a 1D list
flat_list = []
for lst in results:
    for element in lst:
        flat_list.append(element)
print(len(flat_list))
print(len(set(flat_list)))

271
235


In [52]:
flat_list

['https://www.gardeningetc.com/us/buying-guides/best-indoor-plants',
 'https://www.thesill.com/',
 'https://bloomscape.com/product/bamboo-palm/?attribute_pa_pot=slate&utm_source=Google%20Shopping&utm_campaign=Google%20Shopping%20Feed&utm_medium=cpc&utm_term=176756',
 'https://urbanstems.com/products/plants/the-phoebe/NF-PLNT-00049.html',
 'https://www.plants.com/c/house-plants',
 'https://www.homedepot.com/b/Outdoors-Garden-Center-Indoor-Plants/Prepotted/N-5yc1vZc8rnZ1z17or4',
 'https://www.fast-growing-trees.com/products/meyer-lemon-tree?variant=13940798750772&utm_source=google_shopping&utm_medium=organic',
 'https://www.homedepot.com/b/Outdoors-Garden-Center-Indoor-Plants/Palm/N-5yc1vZc8rnZ1z1bq8n',
 'https://www.amazon.com/Houseplants-Plants-Containers-Planters-Pets/dp/B09K9GW3BH?source=ps-sl-shoppingads-lpcontext&psc=1',
 'https://shopgreendigs.com/products/lemon-lime-dracaena-plant?variant=39813428052013&currency=USD',
 'https://www.homedepot.com/b/Outdoors-Garden-Center-Indoor-Pl

In [61]:
#Adding values to the database
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='MY_CUSTOM_BOT',
                                         user='root',
                                         password='Soyunaloca1')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        sql = "INSERT INTO URLS_TABLE(ID, LINK ) VALUES (%s, %s)"
        for i in range(len(flat_list)):
            cursor.execute(sql, (i, flat_list[i],))
            #cursor.commit() 
            connection.commit()
            

except Error as e:
    print("Error while connecting to MySQL", e)
    
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.28
You're connected to database:  ('my_custom_bot',)
MySQL connection is closed
